<a href="https://colab.research.google.com/github/SantanaC4/pos_tagging_treebank/blob/main/pos_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de um Part of Speech Tagging com base no Treebank corpus



In [ ]:
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/SantanaC4/pos_tagging_treebank/main/Sec0-18_training'
df = pd.read_csv(url, header=None, sep='\n')

spliting =  [i.split(" ") for i in df[0]]
extracting_pair = []
for i in spliting:
  if (len(i) == 1):
    extracting_pair.append(tuple(i[0].split("_")))
  else:
    for j in i:
      if (j != ''):
        extracting_pair.append(tuple(j.split("_")))
extracting_pair[0][0]+extracting_pair[0][1]

'PierreNNP'

In [ ]:
## Default dictionary 
## Assign default value to key that is not in dictionary
from collections import defaultdict

## Tallying of tag
counts = defaultdict(int)

for (word, tag) in extracting_pair:
  counts[tag] += 1
counts.items()

## Unigram Tagger
pos = defaultdict(list)
for (word, tag) in extracting_pair:
  pos[word].append(tag)

aux = defaultdict(int)
tags_freq = {}
for (word, tags) in pos.items():
  for i in tags:
    aux[i] += 1
  tags_freq[word] = sorted([(value, key) for (key, value) in aux.items()], reverse=True)
  ## tags_freq[Ex:'offered'] = [(108, 'VBN'), (75, 'VBD'), (4, 'JJ')]
  aux = defaultdict(int)



  
  


{'Pierre': [(6, 'NNP')],
 'Vinken': [(2, 'NNP')],
 ',': [(46522, ',')],
 '61': [(25, 'CD')],
 'years': [(1130, 'NNS')],
 'old': [(213, 'JJ')],
 'will': [(2962, 'MD'), (19, 'NN'), (1, 'VB')],
 'join': [(40, 'VB'), (1, 'VBP')],
 'the': [(39517, 'DT'),
  (7, 'JJ'),
  (6, 'NNP'),
  (1, 'VBP'),
  (1, 'NN'),
  (1, 'CD')],
 'board': [(297, 'NN')],
 'as': [(3355, 'IN'), (699, 'RB'), (1, 'JJ')],
 'a': [(18446, 'DT'),
  (10, 'SYM'),
  (7, 'FW'),
  (2, 'NNP'),
  (2, 'LS'),
  (2, 'JJ'),
  (1, 'IN')],
 'nonexecutive': [(6, 'JJ')],
 'director': [(309, 'NN')],
 'Nov.': [(234, 'NNP'), (3, 'NN'), (1, 'VB')],
 '29': [(74, 'CD')],
 '.': [(37453, '.')],
 'Mr.': [(3856, 'NNP')],
 'is': [(6733, 'VBZ'), (2, 'NNS')],
 'chairman': [(429, 'NN')],
 'of': [(22100, 'IN'), (2, 'RP'), (2, 'RB')],
 'Elsevier': [(1, 'NNP')],
 'N.V.': [(13, 'NNP')],
 'Dutch': [(15, 'JJ'), (8, 'NNP'), (2, 'NNPS')],
 'publishing': [(34, 'NN'), (14, 'VBG')],
 'group': [(603, 'NN'), (1, 'VB')],
 'Rudolph': [(8, 'NNP')],
 'Agnew': [(3, 'NNP